# 期末報告

Group 1 <br>
M114020022 高雲揚 <br>
M114020038 李畇彤 <br>
M114020052 王渙均 <br>
M114020060 陳侑萱 

影片連結: https://www.youtube.com/watch?v=ZU3CyE1Le6c

## 分析動機與目的
+ 因為日前剛解封，出國旅遊的人數大幅增加，尤其是日韓等亞洲地區更是大家出遊的首選，因此我們想針對日本旅遊，查看PTT版上的網民們對日本的哪個景點討論的最熱烈

## 1. 引入套件

In [1]:
from datetime import datetime
from collections import Counter

import jieba
import jieba.analyse
import pandas as pd
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from matplotlib.colors import Colormap
import math
import re
from nltk import ngrams
from nltk import FreqDist
import networkx as nx
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from numpy.linalg import norm
from collections import Counter, namedtuple

In [2]:
# 設定圖的字體
plt.rcParams['font.size'] = 14
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

## 2. 資料前處理

### 2.1 基本資料介紹
資料來源：
+ 工作流程平台蒐集PTT 日旅版文章
+ 關鍵字:「東京、大阪、九州、橫濱、北海道、札幌、仙台」
+ 時間: 2022-03-01 ~ 2023-02-28
+ 資料筆數：共 3081 篇文章

In [4]:
#匯入資料
df = pd.read_csv('rawData/noh_ptt.csv')
df

,system_id,artUrl,artTitle,artDate,artPoster,artCatagory,artContent,artComment,e_ip,insertedDate,dataSource
0,1,https://www.ptt.cc/bbs/TaiwanDrama/M.167841746...,[情報]《模仿犯 CopycatKiller》正式預告,2023-03-10 11:03:59,sodabubble,TaiwanDrama,《模仿犯 Copycat Killer》正式預告\r\nhttps://fb.com/Cop...,"[{""cmtStatus"": ""推"", ""cmtPoster"": ""aax"", ""cmtCo...",61.224.18.162,2023-03-11 02:28:46,ptt
1,2,https://www.ptt.cc/bbs/TaiwanDrama/M.168007635...,[新聞]最帥檢察官！柯佳嬿甜讚「木村慷仁」吳慷仁陰影夢到兇手,2023-03-29 15:52:36,book8685,TaiwanDrama,最帥檢察官！柯佳嬿甜讚「木村慷仁」 吳慷仁陰影夢到兇手\r\nettoday 記者林奕如／台...,"[{""cmtStatus"": ""推"", ""cmtPoster"": ""ysy2003"", ""c...",36.229.81.17,2023-03-30 02:16:02,ptt
2,3,https://www.ptt.cc/bbs/TaiwanDrama/M.168024673...,[情報]Netflix模仿犯10集上線了,2023-03-31 15:12:12,ashin0709,TaiwanDrama,Netflix台劇《模仿犯》將在2023年3月31上線！ 由吳慷仁、柯佳嬿、林心如、姚淳耀等...,"[{""cmtStatus"": ""推"", ""cmtPoster"": ""Parsons25"", ...",42.74.158.222,2023-04-01 02:11:44,ptt
3,4,https://www.ptt.cc/bbs/TaiwanDrama/M.168024836...,[新聞]《模仿犯》從日本小說改編為台劇——如何,2023-03-31 15:39:25,zkow,TaiwanDrama,《模仿犯》從日本小說改編為台劇——如何在類型裡，找到與當代社會的連結？\r\n\r\nVER...,"[{""cmtStatus"": ""推"", ""cmtPoster"": ""for5566"", ""c...",123.192.240.141,2023-04-01 02:11:44,ptt
4,5,https://www.ptt.cc/bbs/TaiwanDrama/M.168027326...,[模仿]兇手是?跟演員表現,2023-03-31 22:34:20,book8685,TaiwanDrama,肯定有雷\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r...,"[{""cmtStatus"": ""→"", ""cmtPoster"": ""hyk99885ggb""...",36.229.81.17,2023-04-01 02:11:44,ptt
...,...,...,...,...,...,...,...,...,...,...,...
93,94,https://www.ptt.cc/bbs/TaiwanDrama/M.168353125...,[心得]模仿犯(有雷),2023-05-08 15:34:14,isafndxuv,TaiwanDrama,防雷\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n...,"[{""cmtStatus"": ""推"", ""cmtPoster"": ""llsr5566"", ""...",223.137.134.160,2023-05-09 02:11:43,ptt
94,95,https://www.ptt.cc/bbs/TaiwanDrama/M.168386113...,[心得]人選是政治還是女性議題？主軸失衡的作品,2023-05-12 11:12:12,ba328585,TaiwanDrama,首先來聊大前提，我是台通的聽眾，所以我有聽到導演在專訪中提到，這部戲其實是個職人\r\n劇，...,"[{""cmtStatus"": ""推"", ""cmtPoster"": ""MrSherlock"",...",59.127.199.80,2023-05-13 02:26:48,ptt
95,96,https://www.ptt.cc/bbs/TaiwanDrama/M.168394981...,[情報]《模仿犯》《人選之人》社群聲量分析,2023-05-13 11:50:06,filmwalker,TaiwanDrama,節錄自 2023第一季 《黑暗榮耀》《重啟人生》《模仿犯》社群聲量分析\r\n ...,"[{""cmtStatus"": ""→"", ""cmtPoster"": ""tinmar"", ""cm...",219.71.217.171,2023-05-14 02:23:24,ptt
96,97,https://www.ptt.cc/bbs/TaiwanDrama/M.168397750...,[Live]不良執念清除師EP11-EP12大結局,2023-05-13 19:31:37,solando,TaiwanDrama,本篇是跟著愛奇藝最新進度的討論\r\n還沒看的朋友可以趕快去追或是避雷一下\r\n\r\n\...,"[{""cmtStatus"": ""推"", ""cmtPoster"": ""ovalcircle1""...",101.12.114.4,2023-05-14 02:23:24,ptt


### 2.2 資料初步清理
清除內文為空值的文章<br>
透過觀察發現ptt的文章常用兩個換行符號作為句子的結尾，因此將兩個換行取代為句號，並清除不重要的字如換行、網址

In [5]:
clear_df = df.copy()

#去除有包含空值的筆數
clear_df.dropna(axis=0, how='any')

#用'。'取代'\n\n'，並移除'\n'
clear_df = clear_df.replace(r'\\n\\n', '。', regex=True).replace(r'\n', '', regex=True)

#移除內文中的網址
clear_df['artContent'] = clear_df['artContent'].replace(r'http\S+', '', regex=True).replace(r'www\S+', '', regex=True)

clear_df

,system_id,artUrl,artTitle,artDate,artPoster,artCatagory,artContent,artComment,e_ip,insertedDate,dataSource
0,1,https://www.ptt.cc/bbs/TaiwanDrama/M.167841746...,[情報]《模仿犯 CopycatKiller》正式預告,2023-03-10 11:03:59,sodabubble,TaiwanDrama,《模仿犯 Copycat Killer》正式預告\r\r\r正體中文(影片上傳：Netfli...,"[{""cmtStatus"": ""推"", ""cmtPoster"": ""aax"", ""cmtCo...",61.224.18.162,2023-03-11 02:28:46,ptt
1,2,https://www.ptt.cc/bbs/TaiwanDrama/M.168007635...,[新聞]最帥檢察官！柯佳嬿甜讚「木村慷仁」吳慷仁陰影夢到兇手,2023-03-29 15:52:36,book8685,TaiwanDrama,最帥檢察官！柯佳嬿甜讚「木村慷仁」 吳慷仁陰影夢到兇手\rettoday 記者林奕如／台北報...,"[{""cmtStatus"": ""推"", ""cmtPoster"": ""ysy2003"", ""c...",36.229.81.17,2023-03-30 02:16:02,ptt
2,3,https://www.ptt.cc/bbs/TaiwanDrama/M.168024673...,[情報]Netflix模仿犯10集上線了,2023-03-31 15:12:12,ashin0709,TaiwanDrama,Netflix台劇《模仿犯》將在2023年3月31上線！ 由吳慷仁、柯佳嬿、林心如、姚淳耀等...,"[{""cmtStatus"": ""推"", ""cmtPoster"": ""Parsons25"", ...",42.74.158.222,2023-04-01 02:11:44,ptt
3,4,https://www.ptt.cc/bbs/TaiwanDrama/M.168024836...,[新聞]《模仿犯》從日本小說改編為台劇——如何,2023-03-31 15:39:25,zkow,TaiwanDrama,《模仿犯》從日本小說改編為台劇——如何在類型裡，找到與當代社會的連結？\r\rVERSE\r...,"[{""cmtStatus"": ""推"", ""cmtPoster"": ""for5566"", ""c...",123.192.240.141,2023-04-01 02:11:44,ptt
4,5,https://www.ptt.cc/bbs/TaiwanDrama/M.168027326...,[模仿]兇手是?跟演員表現,2023-03-31 22:34:20,book8685,TaiwanDrama,肯定有雷\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r [...,"[{""cmtStatus"": ""→"", ""cmtPoster"": ""hyk99885ggb""...",36.229.81.17,2023-04-01 02:11:44,ptt
...,...,...,...,...,...,...,...,...,...,...,...
93,94,https://www.ptt.cc/bbs/TaiwanDrama/M.168353125...,[心得]模仿犯(有雷),2023-05-08 15:34:14,isafndxuv,TaiwanDrama,防雷\r\r\r\r\r\r\r\r\r\r\r\r\r在朋友的推薦下開始看\r終於在前幾天...,"[{""cmtStatus"": ""推"", ""cmtPoster"": ""llsr5566"", ""...",223.137.134.160,2023-05-09 02:11:43,ptt
94,95,https://www.ptt.cc/bbs/TaiwanDrama/M.168386113...,[心得]人選是政治還是女性議題？主軸失衡的作品,2023-05-12 11:12:12,ba328585,TaiwanDrama,首先來聊大前提，我是台通的聽眾，所以我有聽到導演在專訪中提到，這部戲其實是個職人\r劇，從選...,"[{""cmtStatus"": ""推"", ""cmtPoster"": ""MrSherlock"",...",59.127.199.80,2023-05-13 02:26:48,ptt
95,96,https://www.ptt.cc/bbs/TaiwanDrama/M.168394981...,[情報]《模仿犯》《人選之人》社群聲量分析,2023-05-13 11:50:06,filmwalker,TaiwanDrama,節錄自 2023第一季 《黑暗榮耀》《重啟人生》《模仿犯》社群聲量分析\r 同場...,"[{""cmtStatus"": ""→"", ""cmtPoster"": ""tinmar"", ""cm...",219.71.217.171,2023-05-14 02:23:24,ptt
96,97,https://www.ptt.cc/bbs/TaiwanDrama/M.168397750...,[Live]不良執念清除師EP11-EP12大結局,2023-05-13 19:31:37,solando,TaiwanDrama,本篇是跟著愛奇藝最新進度的討論\r還沒看的朋友可以趕快去追或是避雷一下\r\r\r好快就大結...,"[{""cmtStatus"": ""推"", ""cmtPoster"": ""ovalcircle1""...",101.12.114.4,2023-05-14 02:23:24,ptt


### 2.3 斷句
中文主要以標點符號進行斷句，並清除空的字串

In [6]:
clear_df['artContent'] = clear_df['artContent'].str.split("[,，。！!？?]{1,}")

sent_df = clear_df.explode('artContent').reset_index(drop=True)

# 把長度小於1的content刪掉
sent_df = sent_df[sent_df["artContent"].str.len() > 1]

sent_df

,system_id,artUrl,artTitle,artDate,artPoster,artCatagory,artContent,artComment,e_ip,insertedDate,dataSource
0,1,https://www.ptt.cc/bbs/TaiwanDrama/M.167841746...,[情報]《模仿犯 CopycatKiller》正式預告,2023-03-10 11:03:59,sodabubble,TaiwanDrama,《模仿犯 Copycat Killer》正式預告\r\r\r正體中文(影片上傳：Netfli...,"[{""cmtStatus"": ""推"", ""cmtPoster"": ""aax"", ""cmtCo...",61.224.18.162,2023-03-11 02:28:46,ptt
1,1,https://www.ptt.cc/bbs/TaiwanDrama/M.167841746...,[情報]《模仿犯 CopycatKiller》正式預告,2023-03-10 11:03:59,sodabubble,TaiwanDrama,\rNetflix獨家華語影集《模仿犯》改編自日本推理女王宮部美幸同名暢銷小說,"[{""cmtStatus"": ""推"", ""cmtPoster"": ""aax"", ""cmtCo...",61.224.18.162,2023-03-11 02:28:46,ptt
2,1,https://www.ptt.cc/bbs/TaiwanDrama/M.167841746...,[情報]《模仿犯 CopycatKiller》正式預告,2023-03-10 11:03:59,sodabubble,TaiwanDrama,故事描述\r90年代,"[{""cmtStatus"": ""推"", ""cmtPoster"": ""aax"", ""cmtCo...",61.224.18.162,2023-03-11 02:28:46,ptt
3,1,https://www.ptt.cc/bbs/TaiwanDrama/M.167841746...,[情報]《模仿犯 CopycatKiller》正式預告,2023-03-10 11:03:59,sodabubble,TaiwanDrama,一名擅長偵辦兇殺刑案的檢察官郭曉其（吳慷仁 飾）面對當時第一起的連續殺\r人命案,"[{""cmtStatus"": ""推"", ""cmtPoster"": ""aax"", ""cmtCo...",61.224.18.162,2023-03-11 02:28:46,ptt
4,1,https://www.ptt.cc/bbs/TaiwanDrama/M.167841746...,[情報]《模仿犯 CopycatKiller》正式預告,2023-03-10 11:03:59,sodabubble,TaiwanDrama,以他心中衡量正義的那把尺對抗連續殺人案兇手,"[{""cmtStatus"": ""推"", ""cmtPoster"": ""aax"", ""cmtCo...",61.224.18.162,2023-03-11 02:28:46,ptt
...,...,...,...,...,...,...,...,...,...,...,...
4103,98,https://www.ptt.cc/bbs/TaiwanDrama/M.168456187...,[新聞]《大愛劇場：搜尋者》庹宗華、李又汝、,2023-05-20 13:51:04,sodabubble,TaiwanDrama,每集有三個單元,"[{""cmtStatus"": ""推"", ""cmtPoster"": ""lupins"", ""cm...",36.232.24.170,2023-05-21 02:37:09,ptt
4104,98,https://www.ptt.cc/bbs/TaiwanDrama/M.168456187...,[新聞]《大愛劇場：搜尋者》庹宗華、李又汝、,2023-05-20 13:51:04,sodabubble,TaiwanDrama,四條線\r的脈絡同時運作,"[{""cmtStatus"": ""推"", ""cmtPoster"": ""lupins"", ""cm...",36.232.24.170,2023-05-21 02:37:09,ptt
4105,98,https://www.ptt.cc/bbs/TaiwanDrama/M.168456187...,[新聞]《大愛劇場：搜尋者》庹宗華、李又汝、,2023-05-20 13:51:04,sodabubble,TaiwanDrama,即將於6月29日在大愛電視頻道上檔,"[{""cmtStatus"": ""推"", ""cmtPoster"": ""lupins"", ""cm...",36.232.24.170,2023-05-21 02:37:09,ptt
4106,98,https://www.ptt.cc/bbs/TaiwanDrama/M.168456187...,[新聞]《大愛劇場：搜尋者》庹宗華、李又汝、,2023-05-20 13:51:04,sodabubble,TaiwanDrama,YouTube及Yahoo TV同步播出,"[{""cmtStatus"": ""推"", ""cmtPoster"": ""lupins"", ""cm...",36.232.24.170,2023-05-21 02:37:09,ptt


### 2.4 斷詞
斷詞引擎設定：使用 Jieba 當作斷詞系統。
Jieba原先內建為簡體字字典，但我們可以使用繁體字典來做為替換，結果會比較好。另外可能還有一些專有名詞是沒有被包含在預設字典當中，我們也可以使用自定義字典的方式加入，或者使用參數的形式帶入。

In [ ]:
# 初始化斷詞引擎：不一定要設定，只使用內建效果也可以
# 參考網址：https://raw.githubusercontent.com/ldkrsi/jieba-zh_TW/master/jieba/dict.txt
# 將user_dict改為以下字詞陳吉仲、好市多、好事多、產蛋率、吉仲
jieba.set_dictionary('../dict/dict.txt')
jieba.load_userdict('dict/user_dict.txt')